<a href="https://colab.research.google.com/github/dringtech/index-of-transport-accessibility/blob/main/calculate_index_of_transport_accessibility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate Index of Transport Accessibility

This workbook attempts to calculate an index of transport accessibility for each MSOA in the provided dataset.

You will need to prepare your working environment by running the data preparation script.

## Setup

Install dependencies. Depending on where you run this, you might need to add to this list.


In [ ]:
!pip install pyproj progress_tracker

Import modules

In [ ]:
import os
import re
import json
from functools import partial

from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import transform
import pyproj
from pyproj import CRS
import pandas as pd
from progress_tracker import track_progress

Set the working and data directory.

In [ ]:
working_dir = '/content/drive/MyDrive/index-of-transport-accessibility'
os.chdir(working_dir)
geojson_dir = 'data/NorthernIsochrones'

## Projection

The GeoJSON files have coordinates expressed in the WGS84 standard. This means that area would be calculated in degrees squared, which is not that useful for our purposes - although it's a fairly good proxy for the properly projected area.

We need to set up a conversion from WGS84 (EPSG:4326) to OSGB36 (EPSG:27700) for later use.

In [ ]:
proj_from = CRS.from_epsg(4326) # EPSG 4326 is WGS84
proj_to = CRS.from_epsg(27700)  # EPSG 27700 is OSGB
proj = partial(pyproj.transform, proj_from, proj_to)

## Setup files

Get a list of files to process - we're focussing on the 15 minute car journeys for this analysis.

In [ ]:
files = [f for f in os.listdir(geojson_dir) if re.match('.*CAR.*15minutes\.geojson', f)]

Capture the area of each MSOA file by construcing the polygon (including cutouts) and then calculating the area (possibly projected).

In [ ]:
df = pd.DataFrame(data = { 'msoa': [], 'mode': [], 'time': [], 'area': [] })

for filename in track_progress(files, every_n_records=100, every_n_seconds=30):
    # print(".", end='')
    _, _, mode, _, msoa, _, _, _, _, time, *rest = re.split('[\._]', filename)

    with open(os.path.join(geojson_dir, filename)) as file:
        data = json.loads(file.read())

    def make_polygon(c):
        shell, *holes = c
        return Polygon(shell, holes)

    if data['type'] == 'Polygon':
        isochrone = make_polygon(data['coordinates'])

    elif data['type'] == 'MultiPolygon':
        isochrone = MultiPolygon([make_polygon(c) for c in data['coordinates']])

    # It's very slow re-projecting, and areas are broadly correlated if unprojected
    area = transform(proj, isochrone).area
    # area = isochrone.area

    df.loc[len(df.index)] = [msoa, mode, time, area]

Ooof. That took a long time. Let's never do that again if we don't have to... Load the parquet file with `df = pd.read_parquet('data.parquet')`.

In [ ]:
df.to_parquet('raw_data.parquet')

In [ ]:
df = pd.read_parquet('raw_data.parquet')

## Calculate the Index of Transport Accessibility

Calculate the index of transport accessibility by scaling the area of the 15 minute car accessibility to the maximum area. This is to be improved, a lot.

In [ ]:
max_area = max(df['area'])
df['index-of-transport-accessibility'] = (df['area']/max_area).round(3)

Let's plot it, to see what it looks like

In [ ]:
df.sort_values(by=['index-of-transport-accessibility'], ignore_index=True).plot(y=['index-of-transport-accessibility'])

In [ ]:
df[['msoa', 'index-of-transport-accessibility']].sort_values(by=['msoa']).to_csv('accessibility_index.csv', index=False)

# Alternative method

On a hunch, I'm now just looking at the number of MSOAs that can be reached within 60 minutes, regardless of mode of transport

In [ ]:
travel_times = pd.read_csv('data/travel-times.csv').drop(columns=['OriginLatitute', 'OriginLongitude', 'DestinationLatitute', 'DestinationLongitude'])

Calculate the IOTA based on a bare count of MSOAs that can be reached.

In [ ]:
iota = travel_times.loc[travel_times.Minutes <=60].drop(columns=['Mode', 'Minutes']).drop_duplicates().groupby('OriginName').count()
iota.columns = ['index-of-transport-accessibility']
iota.index.name = 'msoa'
reachability = iota.sort_index()

In [ ]:
iota.to_csv('accessibility_index.csv')

There's sort of a correlation, but it's not great

In [ ]:
twin_iotas = df[['msoa', 'index-of-transport-accessibility']].sort_values(by=['msoa']).merge(iota, on='msoa')
twin_iotas.columns = ['msoa', 'area_iota', 'count_iota']
twin_iotas.index = twin_iotas.msoa
twin_iotas = twin_iotas.drop(columns = ['msoa'])
twin_iotas

In [ ]:
twin_iotas.sort_values(by='count_iota').plot()

In [ ]:
twin_iotas.plot.scatter(x='count_iota', y='area_iota')

In [ ]:
def calc_iota(filter):
  r = travel_times.loc[filter].drop(columns=['Mode', 'Minutes']).drop_duplicates().groupby('OriginName').count()
  r.columns = ['index-of-transport-accessibility']
  r.index.name = 'msoa'
  return r

In [ ]:
iota_car = calc_iota(travel_times.Mode == 'CAR')
iota_car.to_csv('accessibility_index_car.csv')

iota_transit = calc_iota(travel_times.Mode == 'TRANSIT,WALK')
iota_transit.to_csv('accessibility_index_transit.csv')

iota_walk = calc_iota(travel_times.Mode == 'WALK')
iota_walk.to_csv('accessibility_index_walk.csv')

iota_walk = calc_iota(travel_times.Mode == 'BICYCLE')
iota_walk.to_csv('accessibility_index_bicycle.csv')